### Imports

In [407]:
import pandas as pd
import numpy as np
from scipy import stats as spstats

from sklearn import model_selection as skms
from sklearn import preprocessing as skpp
from sklearn import metrics as skmet

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# import plotly.graph_objects as go
import plotly.express as px
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline


### Load csv data into pandas dataframe
#### We want our actual feature data to drop the output column

In [408]:
heart_data = pd.read_csv('heart.csv')
heart_feature = heart_data.drop('output', axis = 1)

### Retrieve correlation between columns

In [409]:
heart_data.corr(method = 'pearson')

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
age,1.000000,-0.098447,-0.068653,0.279351,0.213678,0.121308,-0.116211,-0.398522,0.096801,0.210013,-0.168814,0.276326,0.068001,-0.225439
sex,-0.098447,1.000000,-0.049353,-0.056769,-0.197912,0.045032,-0.058196,-0.044020,0.141664,0.096093,-0.030711,0.118261,0.210041,-0.280937
cp,-0.068653,-0.049353,1.000000,0.047608,-0.076904,0.094444,0.044421,0.295762,-0.394280,-0.149230,0.119717,-0.181053,-0.161736,0.433798
trtbps,0.279351,-0.056769,0.047608,1.000000,0.123174,0.177531,-0.114103,-0.046698,0.067616,0.193216,-0.121475,0.101389,0.062210,-0.144931
chol,0.213678,-0.197912,-0.076904,0.123174,1.000000,0.013294,-0.151040,-0.009940,0.067023,0.053952,-0.004038,0.070511,0.098803,-0.085239
fbs,0.121308,0.045032,0.094444,0.177531,0.013294,1.000000,-0.084189,-0.008567,0.025665,0.005747,-0.059894,0.137979,-0.032019,-0.028046
restecg,-0.116211,-0.058196,0.044421,-0.114103,-0.151040,-0.084189,1.000000,0.044123,-0.070733,-0.058770,0.093045,-0.072042,-0.011981,0.137230
thalachh,-0.398522,-0.044020,0.295762,-0.046698,-0.009940,-0.008567,0.044123,1.000000,-0.378812,-0.344187,0.386784,-0.213177,-0.096439,0.421741
exng,0.096801,0.141664,-0.394280,0.067616,0.067023,0.025665,-0.070733,-0.378812,1.000000,0.288223,-0.257748,0.115739,0.206754,-0.436757
oldpeak,0.210013,0.096093,-0.149230,0.193216,0.053952,0.005747,-0.058770,-0.344187,0.288223,1.000000,-0.577537,0.222682,0.210244,-0.430696


### Since it's hard to understand what some of these features are, we add descriptions to each feature
1. age - age in years

2. sex - sex (1 = male; 0 = female)

3. cp - chest pain type (1 = typical angina; 2 = atypical angina; 3 = non-anginal pain; 0 = asymptomatic)

4. trestbps - resting blood pressure (in mm Hg on admission to the hospital)

5. chol - serum cholestoral in mg/dl

6. fbs - fasting blood sugar > 120 mg/dl (1 = true; 0 = false)

7. restecg - resting electrocardiographic results (1 = normal; 2 = having ST-T wave abnormality; 0 = hypertrophy)

8. thalach - maximum heart rate achieved

9. exang - exercise induced angina (1 = yes; 0 = no)

10. oldpeak - ST depression induced by exercise relative to rest

11. slope - the slope of the peak exercise ST segment (2 = upsloping; 1 = flat; 0 = downsloping)

12. ca - number of major vessels (0-3) colored by flourosopy

13. thal - 2 = normal; 1 = fixed defect; 3 = reversable defect

14. num - the predicted attribute - diagnosis of heart disease (angiographic disease status) (Value 0 = < diameter narrowing; Value 1 = > 50% diameter narrowing) 

### Perform a description of the data

In [410]:
heart_data.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [411]:
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


### The data looks clean (no nulls and no object as the dtype)
So we plot the outputs

In [412]:
px.histogram(heart_data, x = 'output', color = 'output', template = 'plotly_dark')

In [413]:
px.histogram(heart_data, x = 'age', color = 'output', template = 'plotly_dark')

In [414]:
px.histogram(heart_data, x = 'chol', color = 'output', template = 'plotly_dark')

### Heatmap of the correlation (We do not necessarily perform feature filtering based on correlation)

In [415]:
# FIGURE OUT HOW TO DO THIS IN PLOTLY BECAUSE PLOTLY IS BEST YESSIR!
px.imshow(abs(heart_data.corr()))


### Box plot the data

In [416]:
for key in heart_feature.columns:
    fig = px.box(heart_feature, y = key)
    fig.show()

### We can also specifically find which features have outliers
We show the IQR for each feature  
And find the outliers using IQR
  

Some of these are unreasonable, as the initial data is boolean-like, so naturally there really are no outliers

In [417]:
for key in heart_feature.columns:
    fq, tq = np.quantile(heart_feature[key], [0.25, 0.75])
    iqr = tq - fq
    outliers = heart_feature[(heart_feature[key] < fq - 1.5 * iqr) | (heart_feature[key] > tq + 1.5 * iqr)]
    if (not outliers.empty):
        print(f'{key}:')
        display(outliers)


trtbps:


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3
101,59,1,3,178,270,0,0,145,0,4.2,0,0,3
110,64,0,0,180,325,0,1,154,1,0.0,2,0,2
203,68,1,2,180,274,1,0,150,1,1.6,1,0,3
223,56,0,0,200,288,1,0,133,1,4.0,0,2,3
241,59,0,0,174,249,0,1,143,1,0.0,1,0,2
248,54,1,1,192,283,0,0,195,0,0.0,2,1,3
260,66,0,0,178,228,1,1,165,1,1.0,1,2,3
266,55,0,0,180,327,0,2,117,1,3.4,1,0,2


chol:


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
28,65,0,2,140,417,1,0,157,0,0.8,2,1,2
85,67,0,2,115,564,0,0,160,0,1.6,1,0,3
96,62,0,0,140,394,0,0,157,0,1.2,1,0,2
220,63,0,0,150,407,0,0,154,0,4.0,1,3,3
246,56,0,0,134,409,0,0,150,1,1.9,1,2,3


fbs:


...

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
101,59,1,3,178,270,0,0,145,0,4.2,0,0,3
204,62,0,0,160,164,0,0,145,0,6.2,0,3,3
221,55,1,0,140,217,0,1,111,1,5.6,0,0,3
250,51,1,0,140,298,0,1,122,1,4.2,1,3,3
291,58,1,0,114,318,0,2,140,0,4.4,0,3,1


caa:


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
52,62,1,2,130,231,0,1,146,0,1.8,1,3,3
92,52,1,2,138,223,0,1,169,0,0.0,2,4,2
97,52,1,0,108,233,1,1,147,0,0.1,2,3,3
99,53,1,2,130,246,1,0,173,0,0.0,2,3,2
158,58,1,1,125,220,0,1,144,0,0.4,1,4,3
163,38,1,2,138,175,0,1,173,0,0.0,2,4,2
164,38,1,2,138,175,0,1,173,0,0.0,2,4,2
165,67,1,0,160,286,0,0,108,1,1.5,1,3,2
181,65,0,0,150,225,0,0,114,0,1.0,1,3,3
191,58,1,0,128,216,0,0,131,1,2.2,1,3,3


thall:


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
48,53,0,2,128,216,0,0,115,0,0.0,2,0,0
281,52,1,0,128,204,1,1,156,1,1.0,1,0,0


We follow the IQR findings by using the z-score method

In [418]:
for key in heart_feature.columns:
    outliers = heart_feature[spstats.zscore(heart_feature[key]) > 3]
    if (not outliers.empty):
        print(f'{key}:\n')
        display(outliers)

trtbps:



,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
223,56,0,0,200,288,1,0,133,1,4.0,0,2,3
248,54,1,1,192,283,0,0,195,0,0.0,2,1,3


chol:



,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
28,65,0,2,140,417,1,0,157,0,0.8,2,1,2
85,67,0,2,115,564,0,0,160,0,1.6,1,0,3
220,63,0,0,150,407,0,0,154,0,4.0,1,3,3
246,56,0,0,134,409,0,0,150,1,1.9,1,2,3


oldpeak:



,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
204,62,0,0,160,164,0,0,145,0,6.2,0,3,3
221,55,1,0,140,217,0,1,111,1,5.6,0,0,3


caa:



,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
92,52,1,2,138,223,0,1,169,0,0.0,2,4,2
158,58,1,1,125,220,0,1,144,0,0.4,1,4,3
163,38,1,2,138,175,0,1,173,0,0.0,2,4,2
164,38,1,2,138,175,0,1,173,0,0.0,2,4,2
251,43,1,0,132,247,1,0,143,1,0.1,1,4,3


### Since we don't really have a normal distribution, standard scaler does not seem to be the right choice.
Instead we will use the minmaxscaler, or the maxabsscaler

In [419]:
# Create training, validation, and testing datasets
split_ratios = {
    'train': 0.70,
    'validation': 0.2,
    'test': 0.1
}

train, test = skms.train_test_split(heart_data, train_size = split_ratios['train'])
train = train.reset_index(drop = True)
val, test = skms.train_test_split(test, train_size = split_ratios['validation'] / (1 - split_ratios['train']))
val = val.reset_index(drop = True)
test = test.reset_index(drop = True)

In [420]:
display(train)
display(val)
display(test)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,58,0,2,120,340,0,1,172,0,0.0,2,0,2,1
1,48,1,2,124,255,1,1,175,0,0.0,2,2,2,1
2,42,1,3,148,244,0,0,178,0,0.8,2,2,2,1
3,46,0,0,138,243,0,0,152,1,0.0,1,0,2,1
4,41,1,1,135,203,0,1,132,0,0.0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,63,1,0,130,330,1,0,132,1,1.8,2,3,3,0
208,58,0,0,130,197,0,1,131,0,0.6,1,0,2,1
209,56,0,0,200,288,1,0,133,1,4.0,0,2,3,0
210,43,1,0,115,303,0,1,181,0,1.2,1,0,2,1


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,51,1,3,125,213,0,0,125,1,1.4,2,1,2,1
1,67,1,0,125,254,1,1,163,0,0.2,1,2,3,0
2,61,1,3,134,234,0,1,145,0,2.6,1,2,2,0
3,65,0,2,140,417,1,0,157,0,0.8,2,1,2,1
4,44,1,1,130,219,0,0,188,0,0.0,2,0,2,1
5,53,1,0,142,226,0,0,111,1,0.0,2,0,3,1
6,43,1,0,110,211,0,1,161,0,0.0,2,0,3,1
7,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
8,57,1,0,152,274,0,1,88,1,1.2,1,1,3,0
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,43,0,0,132,341,1,0,136,1,3.0,1,0,3,0
1,54,0,2,135,304,1,1,170,0,0.0,2,0,2,1
2,71,0,0,112,149,0,1,125,0,1.6,1,0,2,1
3,64,0,0,130,303,0,1,122,0,2.0,1,2,2,1
4,54,1,2,125,273,0,0,152,0,0.5,0,1,2,1
5,56,1,0,125,249,1,0,144,1,1.2,1,1,2,0
6,63,0,1,140,195,0,1,179,0,0.0,2,2,2,1
7,44,1,2,120,226,0,1,169,0,0.0,2,0,2,1
8,39,1,2,140,321,0,0,182,0,0.0,2,0,2,1
9,60,0,0,150,258,0,0,157,0,2.6,1,2,3,0


In [421]:
# Run minmaxscaler now
mmscaler = skpp.MinMaxScaler()
train= pd.DataFrame(mmscaler.fit_transform(train), columns = train.columns)
val = pd.DataFrame(mmscaler.transform(val), columns = val.columns)
test = pd.DataFrame(mmscaler.transform(test), columns = test.columns)

display(train)
display(val)
display(test)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,0.558140,0.0,0.666667,0.245283,0.756184,0.0,0.5,0.780952,0.0,0.000000,1.0,0.00,0.666667,1.0
1,0.325581,1.0,0.666667,0.283019,0.455830,1.0,0.5,0.809524,0.0,0.000000,1.0,0.50,0.666667,1.0
2,0.186047,1.0,1.000000,0.509434,0.416961,0.0,0.0,0.838095,0.0,0.129032,1.0,0.50,0.666667,1.0
3,0.279070,0.0,0.000000,0.415094,0.413428,0.0,0.0,0.590476,1.0,0.000000,0.5,0.00,0.666667,1.0
4,0.162791,1.0,0.333333,0.386792,0.272085,0.0,0.5,0.400000,0.0,0.000000,0.5,0.00,0.333333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,0.674419,1.0,0.000000,0.339623,0.720848,1.0,0.0,0.400000,1.0,0.290323,1.0,0.75,1.000000,0.0
208,0.558140,0.0,0.000000,0.339623,0.250883,0.0,0.5,0.390476,0.0,0.096774,0.5,0.00,0.666667,1.0
209,0.511628,0.0,0.000000,1.000000,0.572438,1.0,0.0,0.409524,1.0,0.645161,0.0,0.50,1.000000,0.0
210,0.209302,1.0,0.000000,0.198113,0.625442,0.0,0.5,0.866667,0.0,0.193548,0.5,0.00,0.666667,1.0


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,0.395349,1.0,1.000000,0.292453,0.307420,0.0,0.0,0.333333,1.0,0.225806,1.0,0.25,0.666667,1.0
1,0.767442,1.0,0.000000,0.292453,0.452297,1.0,0.5,0.695238,0.0,0.032258,0.5,0.50,1.000000,0.0
2,0.627907,1.0,1.000000,0.377358,0.381625,0.0,0.5,0.523810,0.0,0.419355,0.5,0.50,0.666667,0.0
3,0.720930,0.0,0.666667,0.433962,1.028269,1.0,0.0,0.638095,0.0,0.129032,1.0,0.25,0.666667,1.0
4,0.232558,1.0,0.333333,0.339623,0.328622,0.0,0.0,0.933333,0.0,0.000000,1.0,0.00,0.666667,1.0
5,0.441860,1.0,0.000000,0.452830,0.353357,0.0,0.0,0.200000,1.0,0.000000,1.0,0.00,1.000000,1.0
6,0.209302,1.0,0.000000,0.150943,0.300353,0.0,0.5,0.676190,0.0,0.000000,1.0,0.00,1.000000,1.0
7,0.511628,0.0,0.333333,0.433962,0.593640,0.0,0.0,0.600000,0.0,0.209677,0.5,0.00,0.666667,1.0
8,0.534884,1.0,0.000000,0.547170,0.522968,0.0,0.5,-0.019048,1.0,0.193548,0.5,0.25,1.000000,0.0
9,0.534884,1.0,0.666667,0.528302,0.148410,0.0,0.5,0.800000,0.0,0.258065,1.0,0.00,0.666667,1.0


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,0.209302,0.0,0.000000,0.358491,0.759717,1.0,0.0,0.438095,1.0,0.483871,0.5,0.00,1.000000,0.0
1,0.465116,0.0,0.666667,0.386792,0.628975,1.0,0.5,0.761905,0.0,0.000000,1.0,0.00,0.666667,1.0
2,0.860465,0.0,0.000000,0.169811,0.081272,0.0,0.5,0.333333,0.0,0.258065,0.5,0.00,0.666667,1.0
3,0.697674,0.0,0.000000,0.339623,0.625442,0.0,0.5,0.304762,0.0,0.322581,0.5,0.50,0.666667,1.0
4,0.465116,1.0,0.666667,0.292453,0.519435,0.0,0.0,0.590476,0.0,0.080645,0.0,0.25,0.666667,1.0
5,0.511628,1.0,0.000000,0.292453,0.434629,1.0,0.0,0.514286,1.0,0.193548,0.5,0.25,0.666667,0.0
6,0.674419,0.0,0.333333,0.433962,0.243816,0.0,0.5,0.847619,0.0,0.000000,1.0,0.50,0.666667,1.0
7,0.232558,1.0,0.666667,0.245283,0.353357,0.0,0.5,0.752381,0.0,0.000000,1.0,0.00,0.666667,1.0
8,0.116279,1.0,0.666667,0.433962,0.689046,0.0,0.0,0.876190,0.0,0.000000,1.0,0.00,0.666667,1.0
9,0.604651,0.0,0.000000,0.528302,0.466431,0.0,0.0,0.638095,0.0,0.419355,0.5,0.50,1.000000,0.0


In [422]:
# Split data into x and y (input and output)
x_train = train.iloc[:, :-1]
y_train = train['output']
display(x_train)
display(y_train)

x_val = val.iloc[:, :-1]
y_val = val['output']

x_test = test.iloc[:, :-1]
y_test = test['output']

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,0.558140,0.0,0.666667,0.245283,0.756184,0.0,0.5,0.780952,0.0,0.000000,1.0,0.00,0.666667
1,0.325581,1.0,0.666667,0.283019,0.455830,1.0,0.5,0.809524,0.0,0.000000,1.0,0.50,0.666667
2,0.186047,1.0,1.000000,0.509434,0.416961,0.0,0.0,0.838095,0.0,0.129032,1.0,0.50,0.666667
3,0.279070,0.0,0.000000,0.415094,0.413428,0.0,0.0,0.590476,1.0,0.000000,0.5,0.00,0.666667
4,0.162791,1.0,0.333333,0.386792,0.272085,0.0,0.5,0.400000,0.0,0.000000,0.5,0.00,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,0.674419,1.0,0.000000,0.339623,0.720848,1.0,0.0,0.400000,1.0,0.290323,1.0,0.75,1.000000
208,0.558140,0.0,0.000000,0.339623,0.250883,0.0,0.5,0.390476,0.0,0.096774,0.5,0.00,0.666667
209,0.511628,0.0,0.000000,1.000000,0.572438,1.0,0.0,0.409524,1.0,0.645161,0.0,0.50,1.000000
210,0.209302,1.0,0.000000,0.198113,0.625442,0.0,0.5,0.866667,0.0,0.193548,0.5,0.00,0.666667


0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
207    0.0
208    1.0
209    0.0
210    1.0
211    1.0
Name: output, Length: 212, dtype: float64

### Using LinearSVC model

In [423]:
linsvc_model = LinearSVC()
linsvc_model.fit(x_train, y_train)
linsvc_predict = linsvc_model.predict(x_test)
print(skmet.accuracy_score(linsvc_predict, y_test))

0.8387096774193549


### Using SVC model

In [424]:
svc_model = SVC(kernel = 'rbf')
svc_model.fit(x_train, y_train)
svc_predict = svc_model.predict(x_test)
print(skmet.accuracy_score(svc_predict, y_test))

0.8709677419354839


### Using KNeighborsClassifier model

In [425]:
knc_model = KNeighborsClassifier(n_neighbors = 7, weights = 'distance')
knc_model.fit(x_train, y_train)
knc_predict = knc_model.predict(x_test)
print(skmet.accuracy_score(knc_predict, y_test))


0.8709677419354839
